# IT3103 Assignment 1

### Importing the necessary libraries

In [4]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import os

### Importing the dataset
Imports the datasets to be used when training the model

In [18]:
# Get the file from the link
dataset_URL = 'https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/it3103/datasets/fruits.zip'

# Extract the file into a folder
path_to_zip = tf.keras.utils.get_file('fruits.zip', origin=dataset_URL, extract=True, cache_dir='.')

# Set the path to look for the files
dataset_dir = os.path.join(os.path.dirname(path_to_zip), "fruits_extracted/fruits/train")

### Extracting datasets 

In [25]:
# Defining the path to get the training data
freshApplesDir = os.path.join(dataset_dir, "freshapples")
freshBananaDir = os.path.join(dataset_dir, "freshbanana")
freshOrangesDir = os.path.join(dataset_dir, "freshoranges")
rottenApplesDir = os.path.join(dataset_dir, "rottenapples")
rottenBananaDir = os.path.join(dataset_dir, "rottenbanana")
rottenOrangesDir = os.path.join(dataset_dir, "rottenoranges")

# Test data check
print("Total Fresh Apples: ", len(os.listdir(freshApplesDir)))
print("Total Fresh Banana: ", len(os.listdir(freshBananaDir)))
print("Total Fresh Oranges: ", len(os.listdir(freshOrangesDir)))
print("Total Rotten Apples: ", len(os.listdir(rottenApplesDir)))
print("Total Rotten Banana: ", len(os.listdir(rottenBananaDir)))
print("Total Rotten Oranges: ", len(os.listdir(rottenOrangesDir)))



Total Fresh Apples:  182
Total Fresh Banana:  169
Total Fresh Oranges:  163
Total Rotten Apples:  253
Total Rotten Banana:  238
Total Rotten Oranges:  177
